# Library

In [1]:
pip install selectivesearch

  Preparing metadata (setup.py) ... done
  Created wheel for selectivesearch: filename=selectivesearch-0.4-py3-none-any.whl size=4335 sha256=0dd10aeed2eae7d1b58e3b4e3be2f1d9da631dbb99ddb8c2ad62b17c2cf5c3b3
  Stored in directory: /root/.cache/pip/wheels/0e/49/95/01447a4e0f48a135ac91fbdb1dd2a1c0523e40e29957b383a3
Successfully built selectivesearch
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip uninstall opencv-contrib-python opencv-python --yes

!pip install opencv-contrib-python 

Found existing installation: opencv-contrib-python 4.10.0.84
Uninstalling opencv-contrib-python-4.10.0.84:
  Successfully uninstalled opencv-contrib-python-4.10.0.84
Found existing installation: opencv-python 4.10.0.84
Uninstalling opencv-python-4.10.0.84:
  Successfully uninstalled opencv-python-4.10.0.84
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 MB 21.3 MB/s eta 0:00:00:00:0100:01


In [3]:
import os
import numpy as np
import torch
import torch.nn as nn
import torchvision.models as models
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import imageio.v3 as imageio
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
import pandas as pd
import cv2
import shutil
import json
import yaml
import random
import time
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
import concurrent.futures
import multiprocessing as mp
from PIL import Image, ImageOps
from collections import defaultdict, Counter
from torchvision import transforms
from torchvision.transforms import functional as TF
import torch.optim as optim
import re
import selectivesearch
import torch.optim as optim
from torchvision import models
from torchvision.models import AlexNet_Weights
import matplotlib.patches as mpatches
import torch.nn.functional as F
from sklearn.svm import SVC
from concurrent.futures import ProcessPoolExecutor

# Path

In [15]:
#Output folders and file names
OUT_COCO_JSON_NM = 'COCO_annotations_new.json'
OUT_IMAGE_FLDR_NM = 'images'
OUT_CFG_FLDR_NM = 'YOLO_cfg'
OUT_DATAFRAME_NM = 'xview_labels.parquet'
YAML_NM = 'xview_yolo.yaml'
RANDOM_SEED = 2023

in_dataset_pth = Path('/kaggle/input/our-xview-dataset')
out_dataset_pth = Path('/kaggle/working/')
img_fldr = Path(f'/kaggle/input/our-xview-dataset/{OUT_IMAGE_FLDR_NM}')
cfg_fldr_pth = Path(f'/kaggle/input/our-xview-dataset/{OUT_CFG_FLDR_NM}')

out_data_parquet_pth = in_dataset_pth / OUT_DATAFRAME_NM
coco_json_pth = in_dataset_pth / OUT_COCO_JSON_NM
yolo_yaml_pth = cfg_fldr_pth / YAML_NM
train_txt_pth = cfg_fldr_pth / 'train.txt'
val_txt_pth = cfg_fldr_pth / 'val.txt'
test_txt_pth = cfg_fldr_pth / 'test.txt'

# PROPOSALS
OUT_NUMPY_FLDR_NM = 'proposals'
np_fldr = Path(f'/kaggle/working/{OUT_NUMPY_FLDR_NM}')
OUT_PROPOSALS_FLDR_NM = 'proposals'
prop_fldr = Path(f'/kaggle/working/{OUT_PROPOSALS_FLDR_NM}')
PROP_COCO_JSON_NM = 'proposals.json'
proposals_json = out_dataset_pth / PROP_COCO_JSON_NM

random.seed(RANDOM_SEED)

In [5]:
# Pulizia dell'output per cartelle specifiche
def clean_output(output_dir):
    if output_dir.exists() and output_dir.is_dir():
        for item in output_dir.iterdir():
            if item.is_dir():
                shutil.rmtree(item)  # Rimuove la sotto-cartella
            else:
                item.unlink()  # Rimuove il file
        print(f"Cartella {output_dir} pulita.")
    else:
        print(f"Cartella {output_dir} non trovata. Nessuna azione necessaria.")

# Pulisce la cartella di output prima di avviare il processo
clean_output(out_dataset_pth)
clean_output(prop_fldr)

Cartella /kaggle/working pulita.
Cartella /kaggle/working/proposals non trovata. Nessuna azione necessaria.


# DataLoader

## Region Proposals Generation

In [ ]:
def convert_images_to_npy(img_folder, npy_folder):
    os.makedirs(npy_folder, exist_ok=True)
    img_files = [img for img in os.listdir(img_folder) if img.lower().endswith(('.jpg', '.jpeg', '.png'))]

    for img_name in tqdm(img_files, desc="Converting images to .npy format"):
        img_path = os.path.join(img_folder, img_name)
        # Leggi l'immagine
        image = cv2.imread(img_path)
        # Salva in formato .npy
        npy_path = os.path.join(npy_folder, img_name.split('.')[0] + '.npy')
        np.save(npy_path, image)

# Converti le immagini
convert_images_to_npy(img_fldr, np_fldr)

Converting images to .npy format:  42%|████▏     | 19421/45891 [03:07<05:35, 78.83it/s]

In [12]:
## Region Proposals Generation

def process_single_image(image_data, img_fldr):
    img_id = image_data['id']
    img_name = image_data['file_name']
    img_path = os.path.join(img_fldr, img_name)

    if not os.path.exists(img_path):
        raise ValueError(f"Immagine non trovata nel percorso: {img_path}")

    # Carica l'immagine dal file .npy
    image = np.load(img_path)
    original_height, original_width, _ = image.shape

    # Chiamata alla funzione unificata per generare le proposte
    processed_proposals = generate_and_process_proposals(image, original_width, original_height)

    image_data = {
        "image_id": img_id,
        "file_name": img_name,
        "original_size": [original_width, original_height],
        "proposals": []
    }

    # Prepara le proposte per l'output
    for i, proposal in enumerate(processed_proposals):
        x_min, y_min, x_max, y_max = proposal
        image_data["proposals"].append({
            "proposal_id": i,
            "coordinates": [x_min, y_min, x_max, y_max]
        })

    return image_data

def generate_dataset_proposals(coco_json, img_fldr, output_dir, output_json):
    os.makedirs(output_dir, exist_ok=True)
    all_image_data = []

    # Carica il file JSON di COCO
    with open(coco_json, 'r') as f:
        coco_data = json.load(f)

    image_annotations_map = {}
    for annotation in coco_data['annotations']:
        image_id = annotation['image_id']
        if image_id not in image_annotations_map:
            image_annotations_map[image_id] = []
        image_annotations_map[image_id].append(annotation)

    images_with_annotations = [
        image_data for image_data in coco_data['images'] 
        if image_data['id'] in image_annotations_map and len(image_annotations_map[image_data['id']]) > 0
    ]

    # Usa ProcessPoolExecutor per elaborare le immagini in parallelo
    max_workers = 4  # Regola il numero di processi in base al tuo ambiente
    with concurrent.futures.ProcessPoolExecutor(max_workers=max_workers) as executor:
        # Parallelizza l'elaborazione delle immagini e mostra il progresso con tqdm
        results = list(tqdm(executor.map(process_single_image, images_with_annotations, [img_fldr]*len(images_with_annotations)), 
                           total=len(images_with_annotations), 
                           desc="Processing images"))
        
    with open(output_json, 'w') as json_file:
        json.dump(results, json_file, indent=4)

    print(f"Creato file JSON con le region proposals: {output_json}")

In [7]:
'''
def generate_dataset_proposals(txt_file, dir_name):
  # prendo i path delle immagini e li memorizzo in una lista
   with open(txt_file, 'r') as f:
            image_paths = [line.strip() for line in f.readlines()]

  img_dir = img_dir
  os.makedirs(img_dir, exist_ok=True)

  for index in range(len(image_paths)):
    img_name = os.path.basename(image_paths[index]) #prendo il path dell'immagine da self.image_paths in base all'indice fornito
    img_id = int(img_name.replace('_', '').replace('.jpg', '').replace('img', '')) #ricavo l'id dell'immagine -> non l'ho già fatto nell'init?

    dir_image = os.path.join(dir_name, img_id) #nome della directory che conterrà le region proposals relative all'immagine

    if img_id not in self.image_info: #vedo dal dizionario self.image_info se l'immagine è contenuta nel file COCO
              raise ValueError(f"Immagine {img_name} non trovata nel file COCO")

    img_path = os.path.join(self.img_dir, img_name) #prendo il path completo dell'immagine unendo il path della cartella con il nome.jpg dell'immagine
    if not os.path.exists(img_path): #se il path non esiste allora lo segnalo
              raise ValueError(f"Immagine non trovata nel percorso: {img_path}")

    image = Image.open(img_path).convert('RGB') #apro l'immagine e ne ricavo le dimensioni
    original_width, original_height = image.size

     # GESTIONE DELLE REGION PROPOSALS
     proposals_tensor = generate_region_proposals(image) # image = immagine aperta in formato RGB con la libreria PIL
     #    produce una lista di proposals nel formato (x_min, y_min, x_max, y_max)

     processed_proposals = process_proposals(image_tensor, proposals_tensor) # image_tensor = image dopo la data agumentation
     #    produce le immagine = region proposals relative all'immagine di input

     # salvo le region proposals come immagini in una cartella relativa all'immagine di input
     # - salvataggio in dir_image
     os.makedirs(dir_image, exist_ok=True)

    # Iterare sulle region proposals e salvarle come immagini
    for i, proposal_tensor in enumerate(processed_proposals):
        # Convertire il tensore in immagine PIL (assumendo valori nel range [0, 1])
        proposal_image = Image.fromarray((proposal_tensor.numpy() * 255).astype('uint8'))

        # Generare un nome file unico
        proposal_filename = os.path.join(dir_image, f'proposal_{i:04d}.jpg')

        # Salvare l'immagine
        proposal_image.save(proposal_filename)

        print(f"Salvata proposal {i+1}/{len(processed_proposals)}: {proposal_filename}")

        # Aggiungi il path relativo alla lista
        relative_path = os.path.relpath(proposal_filename, dir_name)  # Path relativo rispetto a dir_name
        all_proposal_paths.append(relative_path)

    # Scrittura di tutti i path relativi in un unico file .txt
    with open(output_txt, 'w') as txt_file:
        for path in all_proposal_paths:
            txt_file.write(f"{path}\n")

    print(f"Creato file TXT con i path relativi di tutte le region proposals: {output_txt}")
'''

'\ndef generate_dataset_proposals(txt_file, dir_name):\n  # prendo i path delle immagini e li memorizzo in una lista\n   with open(txt_file, \'r\') as f:\n            image_paths = [line.strip() for line in f.readlines()]\n\n  img_dir = img_dir\n  os.makedirs(img_dir, exist_ok=True)\n\n  for index in range(len(image_paths)):\n    img_name = os.path.basename(image_paths[index]) #prendo il path dell\'immagine da self.image_paths in base all\'indice fornito\n    img_id = int(img_name.replace(\'_\', \'\').replace(\'.jpg\', \'\').replace(\'img\', \'\')) #ricavo l\'id dell\'immagine -> non l\'ho già fatto nell\'init?\n\n    dir_image = os.path.join(dir_name, img_id) #nome della directory che conterrà le region proposals relative all\'immagine\n\n    if img_id not in self.image_info: #vedo dal dizionario self.image_info se l\'immagine è contenuta nel file COCO\n              raise ValueError(f"Immagine {img_name} non trovata nel file COCO")\n\n    img_path = os.path.join(self.img_dir, img_nam

In [8]:
'''
def generate_region_proposals(image, img_width, img_height): #funzione per la generazione delle region proposals per singola immagine
        img_np = np.array(image) #trasformo l'immagine in un array numpy

        if len(img_np.shape) == 3 and img_np.shape[0] == 3: #porto l'immagine nel formato corretto
            img_np = np.transpose(img_np, (1, 2, 0))  # Da [C, H, W] a [H, W, C]
            
        _, regions = selectivesearch.selective_search(img_np, scale=300, sigma=0.9, min_size=10) #richiamo la funzione di selective search
        #scale: granularità della ricerca (più alto, meno dettagliato) ; 
        #sigma: Standard deviation per il filtro gaussiano usato per la segmentazione ;
        #min_size: Dimensione minima di un segmento nell'algoritmo
        #regions: lista di regioni candidate (proposals).
        # - regione = dizionario che contiene info. -> incluse le coordinate di un rettangolo delimitante (region['rect'])

        #CHECK SULLA PRODUZIONE DELLE REGION PROPOSALS
        if len(regions) == 0:
            print(f"Warning: Nessuna regione proposta generata per immagine con forma {img_np.shape}.")

        candidate_proposals = []
        for region in regions: #per ogni regione nella lista delle regioni candidate
            x, y, w, h = region['rect'] # prendo le coordinate del rettangolo delimitante
            if w > 0 and h > 0 and w >= 10 and h >= 10: # prendo solo le regioni con altezza e larghezza >= 10 per evitare che siano molto rumorose
                area = w * h
                x_max, y_max = min(x + w, img_np.shape[1]), min(y + h, img_np.shape[0]) # limito la regione alle dimensioni dell'immagine
                candidate_proposals.append([x, y, x_max, y_max, area]) #inserisco la nuova regione nella lista delle region proposals -> aggiungo un valore in più (area) per facilitare il filtraggio dopo

        unique_proposals = list(set(tuple(p) for p in candidate_proposals)) # converto le proposals in tuple in modo da eliminare i duplicati

        #in questo modo non viene preservata la corrispondenza tra region proposals e labels

        #FILTRO LE PROPOSALS PER PRENDERE SOLO QUELLE UTILI/NECESSARIE
        min_area = 10
        max_area_ratio = 0.8
        proposals = []

        for x_min, y_min, x_max, y_max, area in unique_proposals: #per ogni proposal
            if area >= min_area and area <= max_area_ratio * (img_width * img_height):
                proposals.append((x_min, y_min, x_max, y_max))

        return proposals # restituisce le region proposal valide
'''

'\ndef generate_region_proposals(image, img_width, img_height): #funzione per la generazione delle region proposals per singola immagine\n        img_np = np.array(image) #trasformo l\'immagine in un array numpy\n\n        if len(img_np.shape) == 3 and img_np.shape[0] == 3: #porto l\'immagine nel formato corretto\n            img_np = np.transpose(img_np, (1, 2, 0))  # Da [C, H, W] a [H, W, C]\n            \n        _, regions = selectivesearch.selective_search(img_np, scale=300, sigma=0.9, min_size=10) #richiamo la funzione di selective search\n        #scale: granularità della ricerca (più alto, meno dettagliato) ; \n        #sigma: Standard deviation per il filtro gaussiano usato per la segmentazione ;\n        #min_size: Dimensione minima di un segmento nell\'algoritmo\n        #regions: lista di regioni candidate (proposals).\n        # - regione = dizionario che contiene info. -> incluse le coordinate di un rettangolo delimitante (region[\'rect\'])\n\n        #CHECK SULLA PRODUZI

In [9]:
'''
def process_proposals(image_tensor, proposals, output_size=(227, 227)):  # la funzione trasforma le proposals trovate in immagini ottenute ritagliando l'immagine originale
    processed_proposals = []
    for proposal in proposals:  # per ogni proposal
        try:
            _, H, W = image_tensor.shape  # vedo le dimensioni dell'immagine
            x_min, y_min, x_max, y_max = map(int, proposal)
            x_min, y_min = max(0, x_min), max(0, y_min)
            x_max, y_max = min(W, x_max), min(H, y_max)

            # Controlla se la proposal ha dimensioni valide per l'immagine di partenza -> tecnicamente non si potrebbe eliminare l'if?
            if x_min < x_max and y_min < y_max:
                cropped_region = image_tensor[:, y_min:y_max, x_min:x_max]  # Ritaglio

                # Controlla che il ritaglio non sia vuoto
                if cropped_region.size == 0:
                    print(f"Ritaglio vuoto per proposal: {proposal}. Salto.")
                    continue

                # Controlla che il tensor sia 3D (C, H, W)
                if cropped_region.ndim != 3:
                    print(f"Proposal non valida per il ridimensionamento: {proposal}. Salto.")
                    continue

                # Converti cropped_region in un tensore PyTorch
                cropped_region = torch.tensor(cropped_region).permute(2, 0, 1)  # Cambia il formato da HWC a CHW

                # Ridimensiona la regione proposta
                resized_region = torch.nn.functional.interpolate(
                    cropped_region.unsqueeze(0), size=output_size, mode='bilinear', align_corners=False
                ).squeeze(0)  # Ridimensiona

                processed_proposals.append(resized_region)
        except Exception as e:
            print(f"Errore durante il processamento della proposal: {proposal}. Errore: {e}")

    return processed_proposals  # Lista di tensori delle region proposals
'''

'\ndef process_proposals(image_tensor, proposals, output_size=(227, 227)):  # la funzione trasforma le proposals trovate in immagini ottenute ritagliando l\'immagine originale\n    processed_proposals = []\n    for proposal in proposals:  # per ogni proposal\n        try:\n            _, H, W = image_tensor.shape  # vedo le dimensioni dell\'immagine\n            x_min, y_min, x_max, y_max = map(int, proposal)\n            x_min, y_min = max(0, x_min), max(0, y_min)\n            x_max, y_max = min(W, x_max), min(H, y_max)\n\n            # Controlla se la proposal ha dimensioni valide per l\'immagine di partenza -> tecnicamente non si potrebbe eliminare l\'if?\n            if x_min < x_max and y_min < y_max:\n                cropped_region = image_tensor[:, y_min:y_max, x_min:x_max]  # Ritaglio\n\n                # Controlla che il ritaglio non sia vuoto\n                if cropped_region.size == 0:\n                    print(f"Ritaglio vuoto per proposal: {proposal}. Salto.")\n         

In [10]:
import concurrent.futures

def generate_and_process_proposals(image, img_width, img_height):

    # Esegui la selective search per trovare le regioni di interesse (proposals)
    _, regions = selectivesearch.selective_search(img_np, scale=300, sigma=0.9, min_size=10)

    if len(regions) == 0:
        print(f"Warning: Nessuna regione proposta generata per immagine con forma {img_np.shape}.")

    processed_proposals = []  # Lista per le proposte elaborate

    # Pre-filtraggio delle regioni e raccolta delle coordinate delle proposte
    for region in regions:
        x, y, w, h = region['rect']
        
        # Controlla se la regione è abbastanza grande senza calcoli inutili
        area = w * h
        if w >= 10 and h >= 10 and 10 <= area <= 0.8 * (img_width * img_height):
            x_max, y_max = x + w, y + h
            # Aggiungi le coordinate alla lista delle proposte
            processed_proposals.append([x, y, x_max, y_max])

    return processed_proposals  # Restituisce solo le coordinate delle proposte

# Funzione per elaborare un batch di immagini in parallelo
def process_images_in_parallel(images, img_width, img_height):
    # Utilizza concurrent.futures per elaborare più immagini in parallelo
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # La funzione generate_and_process_proposals verrà applicata su ogni immagine
        results = list(executor.map(lambda image: generate_and_process_proposals(image, img_width, img_height), images))
    return results

In [13]:
generate_dataset_proposals(coco_json_pth, img_fldr, prop_fldr, proposals_json)

Processing images:   1%|▏         | 473/32199 [03:10<3:33:12,  2.48it/s]


KeyboardInterrupt: 

## Positive Region Proposals

In [ ]:
# Funzione IoU per calcolare la sovrapposizione
def box_iou(boxes1, boxes2):
    """Calcola la IoU tra due set di bounding boxes."""
    # Espande le dimensioni per il broadcasting
    boxes1 = boxes1.unsqueeze(1)  # (N, 1, 4)
    boxes2 = boxes2.unsqueeze(0)  # (1, M, 4)
    
    # Calcola gli estremi delle intersezioni
    inter_min = torch.max(boxes1[:, :, :2], boxes2[:, :, :2])  # (N, M, 2)
    inter_max = torch.min(boxes1[:, :, 2:], boxes2[:, :, 2:])  # (N, M, 2)
    inter_sizes = (inter_max - inter_min).clamp(min=0)  # Nessuna area negativa
    inter_area = inter_sizes[:, :, 0] * inter_sizes[:, :, 1]  # Area dell'intersezione
    
    # Calcola le aree delle bounding boxes
    boxes1_area = (boxes1[:, :, 2] - boxes1[:, :, 0]) * (boxes1[:, :, 3] - boxes1[:, :, 1])
    boxes2_area = (boxes2[:, :, 2] - boxes2[:, :, 0]) * (boxes2[:, :, 3] - boxes2[:, :, 1])
    
    # Calcola l'area dell'unione
    union_area = boxes1_area + boxes2_area - inter_area
    return inter_area / union_area  # IoU

In [ ]:
def assign_and_save_regions(region_json_path, bbox_json_path, image_dir, output_dir, iou_threshold=0.5):
    """Associa le regioni proposte ai bounding boxes e salva le regioni positive come immagini."""
    # Carica i file JSON
    with open(region_json_path, 'r') as f:
        regions = json.load(f)

    with open(bbox_json_path, 'r') as f:
        bboxes = json.load(f)
    
    # Crea un dizionario per cercare annotations per image_id
    annotations_by_image = {}
    for annot in bboxes["annotations"]:
        img_id = annot["image_id"]
        if img_id not in annotations_by_image:
            annotations_by_image[img_id] = []
        annotations_by_image[img_id].append((annot["bbox"], annot["category_id"]))
    
    # Crea un dizionario per mappare category_id ai nomi delle categorie
    category_mapping = {cat_id: name for cat_id, name in enumerate(bboxes["categories"])}
    
    # Crea la directory di output se non esiste
    os.makedirs(output_dir, exist_ok=True)
    
    train_images = []  # Per tracciare i percorsi delle immagini salvate
    train_labels = []  # Etichette corrispondenti (category_id)
    counter = 0  # Contatore delle immagini salvate
    
    # Per ogni immagine nelle regioni
    for image in regions:
        image_id = image["image_id"]
        file_name = image["file_name"]
        proposals = image["proposals"]
        
        # Ottieni bounding boxes ground-truth e categorie per l'immagine corrente
        gt_data = annotations_by_image.get(image_id, [])
        if not gt_data:
            # Se non ci sono bounding boxes ground-truth, salta l'immagine
            continue
        
        gt_bboxes = torch.tensor([item[0] for item in gt_data], dtype=torch.float32)
        gt_categories = [item[1] for item in gt_data]
        
        # Trasforma proposals in tensori
        proposal_coords = torch.tensor([p["coordinates"] for p in proposals], dtype=torch.float32)
        
        # Calcola la matrice IoU
        iou_matrix = box_iou(proposal_coords, gt_bboxes)
        
        # Identifica le regioni positive (IoU >= soglia)
        max_ious, indices = torch.max(iou_matrix, dim=1)
        positive_indices = torch.nonzero(max_ious >= iou_threshold).squeeze(1)
        
        # Carica l'immagine originale
        image_path = os.path.join(image_dir, file_name)
        original_image = cv2.imread(image_path)
        if original_image is None:
            print(f"Immagine non trovata: {image_path}")
            continue
        
        # Per ogni regione positiva, ritaglia e salva l'immagine
        for idx in positive_indices:
            x_min, y_min, x_max, y_max = proposal_coords[idx].int().tolist()
            cropped = original_image[y_min:y_max, x_min:x_max]
            
            # Ridimensiona a 224x224
            resized = cv2.resize(cropped, (224, 224), interpolation=cv2.INTER_AREA)
            
            # Ottieni l'etichetta della categoria dal bounding box assegnato
            category_id = gt_categories[indices[idx].item()]
            
            # Salva l'immagine
            output_path = os.path.join(output_dir, f"image_{counter:06d}.jpg")
            cv2.imwrite(output_path, resized)
            
            # Aggiorna train_images e train_labels
            train_images.append(output_path)
            train_labels.append(category_id)
            
            counter += 1
    
    return train_images, train_labels

## Custom Dataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, txt_file, img_dir, coco_json_file, aug=False):
        def generate_id(file_name): #prende il nome.jpg di una immagine e restituisce solo l'identificativo senza prefissi e suffissi
            return file_name.replace('_', '').replace('.jpg', '').replace('img', '')

        with open(txt_file, 'r') as f: #salva le region proposals in un file txt con direttamente le info della cartella
            self.image_paths = [line.strip() for line in f.readlines()] #memorizzo i path delle immagini in una lista

        with open(coco_json_file, 'r') as f: #leggo il file .json - contenente (...) - con coco
            coco_data = json.load(f)

        self.image_annotations = {} #dizionario contenente per ogni immagine una lista di categorie di oggetti presebti

        for annotation in coco_data['annotations']: #uso la sezione annotazioni del file .json per ricavare delle info. sulle immagini del dataset
            image_id = annotation['image_id']
            category_id = annotation['category_id'] # lista di category_id = categorie degli oggetti nell'immagine)

            if image_id not in self.image_annotations: #verifico se l'id dell'immagine è già presente nel dizionario
                self.image_annotations[image_id] = []

            self.image_annotations[image_id].append(category_id)

        self.image_info = {
            int(generate_id(image['file_name'])): image['file_name']
            for image in coco_data['images']
        } #dizionario in cui per ogni nome dell'immagine ottenuta da generate_id(file_name) associa il nome.jpg dell'imagine

        self.base_transform = transforms.Compose([
            transforms.Resize((320, 320)),
            transforms.ToTensor(),
        ]) # trasformazione di base da applicare a tutte le immagini

        # lasciare momentaneamente in caso di aggiornamenti futuri
        self.aug_transform = transforms.Compose([
            transforms.Resize((320, 320)),
            transforms.ToTensor(),
        ]) # strasformazione per la data agumentation

        self.aug = aug

    def __len__(self): # ritorna il numero di elementi in self.image_paths -> chi è?
        return len(self.image_paths)

    def __getitem__(self, index):
        img_name = os.path.basename(self.image_paths[index]) #prendo il path dell'immagine da self.image_paths in base all'indice fornito
        img_id = int(img_name.replace('_', '').replace('.jpg', '').replace('img', '')) #ricavo l'id dell'immagine -> non l'ho già fatto nell'init?

        if img_id not in self.image_info: #vedo dal dizionario self.image_info se l'immagine è contenuta nel file COCO
            raise ValueError(f"Immagine {img_name} non trovata nel file COCO")

        img_path = os.path.join(self.img_dir, img_name) #prendo il path completo dell'immagine unendo il path della cartella con il nome.jpg dell'immagine
        if not os.path.exists(img_path): #se il path non esiste allora lo segnalo
            raise ValueError(f"Immagine non trovata nel percorso: {img_path}")

        image = Image.open(img_path).convert('RGB') #apro l'immagine e ne ricavo le dimensioni
        #original_width, original_height = image.size

        if self.aug: #se la variabile self.aug è alta allora applico la self.aug_transform altrimenti la self.base_transform _> HA SENSO? LE DUE FUNZIONI SONO = !!
            image_tensor = self.aug_transform(image)
        else:
            image_tensor = self.base_transform(image)

        #POTREBBE ESSERCI UN PROBLEMA NELLA CORRISPONDENZA TRA LABLES E REGION PROPOSALS -> perchè in proposals_tensor non ci sono tutte le region proposals perchè alcune vengono scartate
        # -> se la corrispondenza è 1 a 1 allora potrebbe convenire dare a _generate_region_proposals(image) sia le labes che l'immagine? -> non è certo perchè
        # in lables c'è una lista di lable ma non viene specificato dove sono localizzate

        # restituisce un dizionario
        return {
            "regions": image_tensor  # region proposals elaborate, una lista di tensori che rappresentano regioni candidate per il rilevamento
        }